# インプライドボラティリティ（Implied Volatility, IV）とは？

インプライドボラティリティ（IV）は、**市場が予想する将来の価格変動の大きさ**を示す指標です。
オプション価格に織り込まれた期待されるボラティリティであり、過去のデータから計算される**ヒストリカルボラティリティ（HV）**とは異なります。



## IVの意味

市場がどれだけ将来の価格変動を予測しているかを示す
実際のボラティリティではなく、市場の期待値である
オプションの市場価格から逆算して求める


## 計算方法

IVは ブラック・ショールズ・モデル（BSM） や バイナミアルツリーモデル などを用いて求めます。

### ブラック・ショールズ方程式

ヨーロピアン・コールオプションの価格 $C$ は、以下の式で与えられる：

$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$

ヨーロピアン・プットオプションの価格 \(P\) は、以下の式で与えられる：

$
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
$

ここで、

$
d_1 = \frac{\ln(S_0 / K) + (r + \frac{1}{2} \sigma^2) T}{\sigma \sqrt{T}}
$

$
d_2 = d_1 - \sigma \sqrt{T}
$

### 変数の説明
- $S_0$ ：現在の株価（原資産価格）
- $K$ ：権利行使価格（行使価格）
- $T$ ：満期までの時間（年）
- $r$ ：無リスク金利（リスクフリーレート）
- $\sigma$ ：ボラティリティ（原資産の標準偏差）
- $N(x)$ ：標準正規分布の累積分布関数（CDF）

**この方程式を 市場のオプション価格を使って 逆算すると IV（σ） が求まります。**


# IVの特徴

✅ IVが高い → 市場がボラティリティの高まりを予測 

✅ IVが低い → 市場が安定すると予測

## 注：
インプライドボラティリティ（IV）は、会社そのものではなく、その会社のオプションに対して計算されます。

したがって、 「ある会社のインプライドボラティリティが高い」＝ その会社の証券のリスクが上がる という解釈は、完全には正しくありません。

ただし、IVが高いということは、市場がその**会社の株価が大きく変動する可能性が高いと考えていること**を意味するので、リスクが上がる可能性はあります。

- 💡 「会社のIVが高い」= その会社のオプション市場が「株価の大きな変動」を予測している
- 💡 「IVが高い」= 必ずしもリスク（下落）とは限らず、上昇の可能性もある

もし「会社のリスクが上がるかどうか」を判断したいなら、ヒストリカルボラティリティ（HV）やベータ値（β） など、他のリスク指標と組み合わせて分析するのがよいでしょう！

## ボラティリティ・スマイル（Volatility Smile）

- IVはアット・ザ・マネー（ATM） で低く、アウト・オブ・ザ・マネー（OTM） で高くなる傾向

## IVと実際のボラティリティは一致しない

- IVが高くても、実際には価格がほとんど動かないこともある（例：IVが上昇しても価格変動が小さい "IV Crush"）

In [16]:
from scipy.stats import norm
import numpy as np
from scipy.optimize import bisect

def black_scholes(S0, K, T, r, sigma, option_type="call"):
    """ブラック・ショールズモデルでオプション価格を計算"""
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
    return price

def implied_volatility(market_price, S0, K, T, r, option_type="call"):
    """市場価格からインプライドボラティリティを計算"""
    def objective(sigma):
        return black_scholes(S0, K, T, r, sigma, option_type) - market_price
    
    # 最小・最大のボラティリティでの価格
    min_price = black_scholes(S0, K, T, r, 0.001, option_type)
    max_price = black_scholes(S0, K, T, r, 5.0, option_type)
    
    # 市場価格が範囲外の場合は NaN を返す
    if market_price < min_price or market_price > max_price:
        print(f"市場価格 {market_price} は {min_price:.2f} 〜 {max_price:.2f} の範囲外です。")
        return np.nan
    
    try:
        iv = bisect(objective, 0.001, 5.0, xtol=1e-6)
        return iv
    except ValueError:
        print("ボラティリティの求解に失敗しました。")
        return np.nan  # 解が見つからない場合

# パラメータ設定
S0 = 100    # 現在の株価
K = 150     # 権利行使価格
T = 1       # 満期 (年)
r = 0.05    # リスクフリーレート
market_price = 10.0  # 市場価格

# インプライドボラティリティ計算
iv = implied_volatility(market_price, S0, K, T, r, option_type="call")
if not np.isnan(iv):
    print(f"インプライドボラティリティ: {iv * 100:.2f}%")


インプライドボラティリティ: 54.84%


In [17]:

# 理論価格をインプライドボラティリティを使って計算
theoretical_price = black_scholes(S0, K, T, r, iv, option_type='call')

# 市場価格と理論価格を比較
print(f"理論価格: {theoretical_price:.2f}")
print(f"市場価格: {market_price:.2f}")

理論価格: 10.00
市場価格: 10.00


✅ まとめ
- IV はブラック・ショールズ式を逆算して求める
- 数値解法（scipy.optimize.bisect）で解く
- IV を使えば、市場のボラティリティ予測やオプションの割高・割安を判断できる

# 数値解法（Numerical Methods）とは

解析的な方法で解けないまたは解きにくい問題を近似的に解くための数学的技法です。これには、方程式や微分方程式の数値的解法、最適化問題、積分の近似などが含まれます。特に金融工学や経済学において、数値解法は非常に重要であり、ブラック-ショールズ方程式やオプション価格の計算でよく使用されます。

インプライドボラティリティ（Implied Volatility）は、オプション価格に基づいて計算されるボラティリティで、実際の市場で取引されているオプションの価格から逆算されます。具体的には、ブラック-ショールズモデルに基づいて、オプションの市場価格と理論価格（ブラック-ショールズモデルを使って計算された価格）の差を最小にするボラティリティを求めます。

ブラック-ショールズ方程式を使ってオプションの理論価格を計算する場合、理論価格はインプライドボラティリティに依存しますが、このボラティリティは解析的に求めることができません。したがって、インプライドボラティリティを求めるために数値解法が用いられます。

たとえば、ブラック-ショールズ方程式において、オプション価格とボラティリティの関係は非線形です。これを解くためには、数値的な方法（ニュートン法や二分法など）を用いて、特定のオプション価格に対するボラティリティを反復的に求めることになります。インプライドボラティリティは、次のようにして求めます：

オプション市場での実際の取引価格を取得。
ブラック-ショールズ方程式を用いて理論価格を計算。
理論価格と市場価格の差を最小化するボラティリティを反復的に求める（数値解法を使用）。
このプロセスは、解析的な解法が存在しないため、数値解法が非常に重要になります。

このコードは、ブラック-ショールズ方程式を用いてオプションの理論価格を計算し、実際の市場価格と比較することで、インプライドボラティリティ（IV）を求めるためのものです。その意義は以下の通りです。

1. オプション価格の理論的な根拠を提供

ブラック-ショールズモデルは、オプション価格を決定するための理論的枠組みを提供します。市場で取引されるオプション価格は、実際には供給と需要によって決まりますが、このコードでは、理論的にどのような価格になるべきかを計算しています。これにより、オプションが市場でどれほど「適正」に取引されているかを分析することができます。

2. インプライドボラティリティ（IV）の求解

インプライドボラティリティ（IV）は、オプション市場の最も重要なパラメータの1つで、オプションの理論価格が市場価格と一致するようなボラティリティです。通常、IVは市場価格から逆算する必要があり、解析的な解法は存在しません。そのため、数値解法（この場合、fsolve）を用いてIVを求めることになります。このプロセスは、ブラック-ショールズ方程式が提供する理論価格と市場価格のギャップを最小化するボラティリティを見つけることを意味します。

3. 市場の過小評価・過大評価を分析
   
インプライドボラティリティを求めることで、市場がオプションのリスクをどれほど評価しているかが明らかになります。具体的には：

- 市場価格が理論価格より高い場合：オプションが過大評価されている可能性があり、投資家は将来の価格変動（ボラティリティ）が非常に大きいと予測しているか、需要が過剰であることを示唆します。
  
- 市場価格が理論価格より低い場合：オプションが過小評価されている可能性があり、投資家は将来の価格変動を過小評価しているか、市場で取引されている価格が低すぎることを示唆します。
このコードを使って、実際に取引されているオプションの価格と理論的に計算された価格を比較し、その差が意味するところを分析できます。

4. オプション市場の効率性の評価

インプライドボラティリティの計算を通じて、オプション市場がどれほど効率的かを評価することができます。理論価格と市場価格に大きな差がある場合、市場は効率的でないと考えられ、そのギャップを取引機会として利用できる場合があります。特に、オプション取引におけるアービトラージ（価格差を利用した無リスク取引）の可能性を分析する際に有用です。

5. リスク管理とポートフォリオ最適化

インプライドボラティリティはオプション取引におけるリスク管理やポートフォリオ最適化に役立ちます。ボラティリティが高ければ、オプションのリスクも大きくなるため、投資家はそのリスクに応じたポートフォリオの調整を行います。IVを正確に求めることで、オプションの価格とリスクの関係をより深く理解でき、効果的なヘッジ戦略を立てることが可能になります。

6. オプション取引における意思決定をサポート

実際のオプション取引において、このコードを使用することで、投資家やトレーダーは市場での取引判断をより明確に行うことができます。インプライドボラティリティを求めることで、市場がどれほどオプションに対するボラティリティを評価しているかを理解し、それに基づいた戦略（例えば、売買タイミングの判断）を構築する手助けとなります。

# 20250206に更新

In [7]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

# ブラック・ショールズ・モデル（Black-Scholes Model）
def black_scholes(S, K, T, r, sigma, option_type="c"):
    """ ブラック・ショールズ式でオプション価格を計算 """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "c":  # コールオプション
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "p":  # プットオプション
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'c' (call) or 'p' (put)")
    
    return price

# インプライドボラティリティ（IV）を求める関数
def implied_volatility(market_price, S, K, T, r, option_type="c"):
    """ オプション市場価格とブラック・ショールズ価格を一致させるボラティリティを求める """
    try:
        return brentq(lambda sigma: black_scholes(S, K, T, r, sigma, option_type) - market_price,
                      0.01, 3.0)  # 初期範囲を1%～300%に設定
    except ValueError:
        return np.nan  # 計算できない場合はNaNを返す

# 取得する銘柄
ticker = "AAPL"

# 株価データの取得
stock = yf.Ticker(ticker)
stock_price = stock.history(period="1d")["Close"].iloc[-1]

# オプションデータの取得（直近の満期日を選択）
expiration = stock.options[0]
opt_chain = stock.option_chain(expiration)
calls = opt_chain.calls  # コールオプション
puts = opt_chain.puts  # プットオプション

# 最も取引が活発なコールオプションを取得
option = calls.sort_values("volume", ascending=False).iloc[0]
strike = option["strike"]
market_price = option["lastPrice"]
T = 30 / 365  # 満期までの日数（30日と仮定）
r = 0.01  # 無リスク金利（1%と仮定）

# インプライドボラティリティ（IV）を計算
iv = implied_volatility(market_price, stock_price, strike, T, r, "c")

print(f"銘柄: {ticker}")
print(f"株価: {stock_price:.2f}, 権利行使価格: {strike}, 市場価格: {market_price}")
print(f"インプライドボラティリティ: {iv:.2%}")


銘柄: AAPL
株価: 232.47, 権利行使価格: 235.0, 市場価格: 1.0
インプライドボラティリティ: 7.30%


# トレードでの活用

🎯 オプション取引

- IVが高いとき → オプション売り（割高だから）
- IVが低いとき → オプション買い（割安だから）

📊 ボラティリティインデックス（VIX）

- VIX は S&P500のオプションのIVを元に計算される指数
- **"恐怖指数"** とも呼ばれ、IVが高まると市場のリスクが増加していると判断される

# VIXの計算手順
1. S&P 500の過去の価格データを取得
2. 日次リターンを計算
3. 標準偏差を求め、年率換算
4. VIXとして出力

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf

# S&P 500のデータを取得
ticker = "^GSPC"  # S&P 500のシンボル
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")

# 終値を使用
data["Return"] = np.log(data["Close"] / data["Close"].shift(1))

# ボラティリティ（標準偏差）を計算（年率換算: 252取引日ベース）
hv = np.std(data["Return"].dropna()) * np.sqrt(252) * 100  # VIXはパーセンテージ表記

print(f"推定VIX（ヒストリカルボラティリティベース）: {hv:.2f}")



[*********************100%***********************]  1 of 1 completed

推定VIX（ヒストリカルボラティリティベース）: 13.07


# 本格的なVIXの計算

**上記の恐怖指数は簡易的なものであり、本来のVIX計算には、以下の手順が必要**です：

1. OTM（アウト・オブ・ザ・マネー）オプションのインプライドボラティリティを収集
2. 加重平均でボラティリティを算出
3. 30日間のボラティリティを年率換算

この計算には、yfinance ではなく CBOE のオプション価格データが必要になります。もし本格的にVIXを計算したい場合は、CBOEのデータAPI や quandl を使う方法も検討してください。